#### Data preprocessing

This code still yield black picture problem

In [ ]:
#normalization from image captioning work
def preprocess_input(x):
  x /= 255.
  x -= 0.5
  x *= 2.
  return x

#preprocessing based on keras documentation and old image captioning model

def preprocess(image_path):
  img = image.load_img(image_path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  return x

plt.imshow(preprocess(wrist_paths[0])[0])
#print(preprocess(wrist_paths[0]).shape)
#images.vstack([images,new])

In [ ]:
#tensorflow code trial for image preprocessing and reading
import tensorflow as tf
imagess = tf.image.decode_png("MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png",channels=3)
resized_image = tf.image.resize_images(imagess, [224, 224])
#resized_image=np.array(resized_image)


reading and resizing images

In [ ]:
#reading images and resizing 
img = image.load_img("MURA-v1.1/train/XR_SHOULDER/patient00001/study1_positive/image1.png", target_size=(224, 224))

In [ ]:
#with no resize
images=[]
for path in tqdm(wrist_paths[:1000]):
    wrist=plt.imread(path)
    images.append(wrist)

images=np.array(images)

print(images.shape)

stack images to keras shape

In [ ]:
#get mean image of array of images for normalization
# append (224,224,3) images to z=(n,224,224,3)
z=[]
z.append(np.array(img))
z=np.array(z)
print(z.shape)
m=np.mean(z,axis=0)
print(m.shape)

#another way to stack images to z
c=np.vstack([x,x])
print(c.shape)

labels processing for keras

In [ ]:
#making labels one hot encoded for "xpected dense_39 to have 4 dimensions, but got array with shape (500, 1)" problem
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit(np.array([wrist_labels]))
Y=enc.fit_transform(np.array([wrist_labels[:500]]), y=None)

In [ ]:
Y=keras.utils.to_categorical(wrist_labels_tr, num_classes=2)
#To make array (n,2)

write custom keras generator:
https://medium.com/@ensembledme/writing-custom-keras-generators-fe815d992c5a
image captioning:
https://colab.research.google.com/drive/1YWGQkDKfMfDvelZnPuWaZatfC7XEEk-h#scrollTo=0IPtsB1noe2B

Keras sequential model
https://keras.io/models/sequential/#fit

In [ ]:
bone_model = Sequential()
bone_model.add(MobileNetV2_model)
bone_model.add(Dense(512, activation='relu'))
bone_model.add(Dense(128, activation='relu'))
bone_model.add(Dense(32, activation='relu'))
#2 because 2 labels
bone_model.add(Dense(2, activation='softmax'))
# Compile model
bone_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
bone_model.summary()

** Inception fine tuning : keras doc**
**Inception code from Keras docs**
https://keras.io/applications/

In [ ]:
#this is done after freezing and training first
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(wrist_images_tr, wrist_labels_tr, epochs=5, validation_data=(wrist_images_val, wrist_labels_val), shuffle=True, verbose=1 )